In [8]:
import os
import cv2
from matplotlib import pyplot as plt
from random import randint
import numpy as np
from math import sqrt

In [9]:
def get_circle_points(center, min_x, max_x, radius):
    a, b = center
    x_lst = [x for x in range(min_x, max_x+1)]
    y_lst = []
    for x in x_lst:
        y = sqrt(radius**2 - (x - a)**2) + b
        y_lst.append(round(y))
    for y in y_lst[::-1]:
        dy = abs(b-y)
        y_lst.append(b - dy)
    x_lst = x_lst + x_lst[::-1]
    return x_lst, y_lst

def get_circle_countour(center, min_x, max_x, radius):
    x_lst, y_lst = get_circle_points(center, min_x, max_x, radius)
    contour = [(x, y) for x, y in zip(x_lst, y_lst)]
    return contour

def random_color(img):
    cp_img = img.copy()
    cp_img = np.array(cp_img, dtype=np.float32)
    randomize = randint(0, 1)
    if randomize:
        rand0 = randint(1, 100) / 100.
        rand1 = randint(1, 100) / 100.
        rand2 = randint(1, 100) / 100.
        cp_img[:, :, 0] *= rand0
        cp_img[:, :, 1] *= rand1
        cp_img[:, :, 2] *= rand2
    return np.array(cp_img, dtype=np.uint8)

def random_slice(img, img_h):
    cp_img = img.copy()
    x_lst = [randint(320, 450)]*img_h
    if randint(0, 1):
        for x_idx in range(0, img_h):
            rand_dx = randint(-15, 15)
            x_lst[x_idx] = x_lst[x_idx] + rand_dx
        for row, x in zip(cp_img, x_lst):
            row[x:] = (0, 0, 0)
    else:
        for row, x in zip(cp_img, x_lst):
            row[x:] = (0, 0, 0)
    return cp_img

def random_line(img):
    if len(img.shape) == 3:
        w, h, _ = img.shape
        get_color = lambda: (randint(0, 255), randint(0, 255), randint(0, 255))
    else:
        w, h = img.shape
        get_color = lambda: randint(0, 255)
        
    line_nums = randint(0, 7)
    line_th = [randint(1, 15) for i in range(line_nums)]
    for i in range(line_nums):
        rand_x1_y1 = (randint(0, w-1), randint(0, h-1))
        rand_x2_y2 = (randint(0, w-1), randint(0, h-1))
        cv2.line(img, rand_x1_y1, rand_x2_y2, get_color(), line_th[i])
    return img

def get_mask(img, circle_contour):
    mask = np.zeros_like(img)
    cv2.drawContours(mask, np.array([circle_contour]), 0, (255, 255, 255), -1)
    out = np.zeros_like(img)
    out[mask == 255] = img[mask == 255]
    return out

def rotateImage(image, angle):
    image_center = tuple(np.array(image.shape[1::-1]) / 2)
    rot_mat = cv2.getRotationMatrix2D(image_center, angle, 1.0)
    result = cv2.warpAffine(image, rot_mat, image.shape[1::-1], flags=cv2.INTER_LINEAR)
    return result

def random_resize(img):
    if len(img.shape) == 3:
        h, w, _ = img.shape
    else:
        h, w = img.shape
    rand_scale = randint(25, 150) / 100.
    w = int(w * rand_scale)
    h = int(h * rand_scale)
    return cv2.resize(img, (w, h))

def random_noise(img):
    cp_img = img.copy() 
    rand_num = randint(0, 4)
    if rand_num == 1:
        cp_img = cp_img + 3 * ball_img.std() * np.random.random(ball_img.shape)
    elif rand_num == 2:
        cp_img[:, :, 0] = cp_img[:, :, 0] + \
                          3 * cp_img[:, :, 0].std() * \
                          np.random.random(cp_img[:, :, 0].shape)
    elif rand_num == 3:
        cp_img[:, :, 1] = cp_img[:, :, 1] + \
                          3 * cp_img[:, :, 1].std() * \
                          np.random.random(cp_img[:, :, 1].shape)
    elif rand_num == 4:
        cp_img[:, :, 2] = cp_img[:, :, 2] + \
                          3 * cp_img[:, :, 2].std() * \
                          np.random.random(cp_img[:, :, 2].shape)
    return cp_img

def random_blur(img):
    rand_num = randint(0, 1)
    if rand_num == 1:
        rand_kernel = randint(3, 8)
        img = img.copy()
        img = cv2.blur(img, (rand_kernel, rand_kernel))
    return img

def crop_ball(img):
    croped_img = img[300-133:300+133, 300-133:300+133]
    resized = random_resize(croped_img)
    return resized

def get_final_mask(img, ball_img):
    h, w, _ = img.shape
    resized = cv2.resize(img, (w*2, h*2))
    ball_size, _, _ = ball_img.shape
    zeros = np.zeros_like(resized)
    
    h, w, _ = resized.shape
    max_x = w - ball_size
    max_y = h - ball_size
    
    rand_x = randint(0, max_x-1)
    rand_y = randint(0, max_y-1)
    
    zeros[rand_y:rand_y+ball_size, rand_x:rand_x+ball_size] = ball_img
    
    gray = cv2.cvtColor(zeros, cv2.COLOR_BGR2GRAY)
    ret, thr = cv2.threshold(gray, 5, 255, cv2.THRESH_BINARY)

    mask = cv2.bitwise_not(thr)
    return mask, zeros
    
def put_ball_on_img(img, mask, zeros):
    res = cv2.bitwise_and(img, img ,mask = mask)
    final = cv2.bitwise_or(res, zeros)
    return final

def get_ball_bounding_box(mask):
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    points = []
    for c in contours:
        for cc in c:
            for ccc in cc:
                points.append(ccc)

    points = points[4:]
    x_lst = [p[0] for p in points]
    y_lst = [p[1] for p in points]

    x_min, y_min = min(x_lst), min(y_lst)
    x_max, y_max = max(x_lst), max(y_lst)
    return (x_min, y_min), (x_max, y_max)

def generate_img_with_ball_and_bounding_box(img, ball_img):
    cp_ball_img = ball_img.copy()
    cp_ball_img = random_color(cp_ball_img)
    cp_ball_img = random_line(cp_ball_img)
    cp_ball_img = random_noise(cp_ball_img)
    cp_ball_img = random_blur(cp_ball_img)
    
    w, h, _ = cp_ball_img.shape
    center = (w//2, h//2)
    radius = 132

    min_x, max_x = center[0] - radius, center[0] + radius

    circle_contour = get_circle_countour(center, min_x, max_x, radius)

    masked = get_mask(cp_ball_img, circle_contour)
    sliced = random_slice(masked, masked.shape[0])
    rotated = rotateImage(sliced, randint(-180, 180))
    croped = crop_ball(rotated)
    final_mask, zeros = get_final_mask(img, croped)
    final = put_ball_on_img(cv2.resize(img, final_mask.shape[::-1]), final_mask, zeros)
    
    bbox = get_ball_bounding_box(final_mask)
    
    return final, bbox

In [10]:
import cv2


data_path = os.path.join(os.getcwd(), "data")
ball_dir = os.path.join(data_path, "blender_ball")
not_ball_dir = os.path.join(data_path, "not_ball")
images_names_lst = os.listdir(ball_dir)
to_save = os.path.join(data_path, "generated_images")

bboxes_dict = {}

for img_name in os.listdir(not_ball_dir):
    full_img_name = os.path.join(not_ball_dir, img_name)
    img = cv2.imread(full_img_name)
    if img is not None:
        try:
            rand_idx = randint(0, len(images_names_lst)-1)
            ball_img_name = os.path.join(ball_dir, images_names_lst[rand_idx])
            ball_img = cv2.imread(ball_img_name)
            result, bbox = generate_img_with_ball_and_bounding_box(img, ball_img)
            cv2.imshow("i", result)
            cv2.waitKey(20)
            cv2.imwrite(os.path.join(os.getcwd(), to_save, img_name), result)
            bboxes_dict[img_name] = bbox
        except:
            pass
cv2.destroyAllWindows()

In [11]:
bboxes_dict

{'1728.jpg': ((302, 170), (680, 512)),
 '95.jpg': ((746, 27), (996, 293)),
 '890.jpg': ((109, 57), (466, 415)),
 '1266.jpg': ((603, 231), (916, 545)),
 '1572.jpg': ((266, 602), (638, 975)),
 '1714.jpg': ((723, 193), (995, 465)),
 '1640.jpg': ((122, 273), (362, 513)),
 '1401.jpg': ((347, 279), (593, 567)),
 '1479.jpg': ((77, 154), (458, 520)),
 '1429.jpg': ((138, 512), (431, 805)),
 '1542.jpg': ((100, 57), (402, 387)),
 '1501.jpg': ((393, 478), (710, 795)),
 '353.jpg': ((763, 95), (952, 310)),
 '1745.jpg': ((589, 398), (709, 518)),
 '38.jpg': ((669, 494), (878, 707)),
 '72.jpg': ((398, 190), (721, 515)),
 '1453.jpg': ((203, 764), (435, 969)),
 '663.jpg': ((754, 158), (959, 350)),
 '185.jpg': ((10, 266), (348, 604)),
 '1331.jpg': ((684, 257), (912, 547)),
 '589.jpg': ((753, 372), (836, 455)),
 '1337.jpg': ((645, 212), (888, 422)),
 '140.jpg': ((326, 243), (514, 451)),
 '1373.jpg': ((190, 147), (353, 271)),
 '229.jpg': ((331, 241), (552, 451)),
 '1763.jpg': ((175, 258), (301, 347)),
 '842

In [12]:
import json

str_bbox = {}

for k, i in bboxes_dict.items():
    x1y1, x2y2 = i
    
    s_x1y1 = (str(x1y1[0]), str(x1y1[1]))
    s_x2y2 = (str(x2y2[0]), str(x2y2[1]))
    
    str_bbox[k] = (s_x1y1, s_x2y2)
    
labels_path = os.path.join(data_path, "labels", "bboxes.json")

json.dump(str_bbox, open(labels_path, 'w' ))

In [18]:
generated_images_dir = to_save
path_to_save_txt_labels_train = os.path.join(data_path, "labels", "train.txt")

with open(path_to_save_txt_labels_train, 'a') as file:
    for ph_name in os.listdir(generated_images_dir)[:750]:
        x1, y1 = bboxes_dict[ph_name][0]
        x2, y2 = bboxes_dict[ph_name][1]
        full_string = os.path.join(generated_images_dir, ph_name)
        full_string = full_string + " {},{},{},{},0\n".format(x1, y1, x2, y2)
        file.write(full_string)

In [20]:
generated_images_dir = to_save
path_to_save_txt_labels_test = os.path.join(data_path, "labels", "test.txt")

with open(path_to_save_txt_labels_test, 'a') as file:
    for ph_name in os.listdir(generated_images_dir)[750:]:
        x1, y1 = bboxes_dict[ph_name][0]
        x2, y2 = bboxes_dict[ph_name][1]
        full_string = os.path.join(generated_images_dir, ph_name)
        full_string = full_string + " {},{},{},{},0\n".format(x1, y1, x2, y2)
        file.write(full_string)


In [25]:
import csv
import os

data_path = os.path.join(os.getcwd(), "data")
path_to_save_txt_labels_test = os.path.join(data_path, "labels", "test.txt")
path_to_save_txt_labels_train = os.path.join(data_path, "labels", "train.txt")


def parse_annotation_line(line):
    file_name, data = line.split(" ")
    min_x, min_y, max_x, max_y, clss = [int(i) for i in data.split(",")]
    height, width = max_y - min_y, max_x - min_x
    return ["golf_ball", file_name, height, width,
            max_x, min_x, max_y, min_y] 

def create_data_dict(fields, data):
    ret_dict = {}
    for f, d in zip(fields, data):
        ret_dict[f] = d
    return ret_dict

data_fileds = ["class", "fileName", "height", "width",
              "xmax", "xmin", "ymax", "ymin"]


train_csv_annotation_file = os.path.join(data_path, "labels", "train.csv")
test_csv_annotation_file = os.path.join(data_path, "labels", "test.csv")


with open(path_to_save_txt_labels_train, "r") as train_annotation:
    lines = [line for line in train_annotation]
    with open(train_csv_annotation_file, 'w', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=data_fileds)
        writer.writeheader()
        for line in lines:
            data = parse_annotation_line(line)
            data_dict = create_data_dict(data_fileds, data)
            writer.writerow(data_dict)
            
with open(path_to_save_txt_labels_test, "r") as test_annotation:
    lines = [line for line in test_annotation]
    with open(test_csv_annotation_file, 'w', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=data_fileds)
        writer.writeheader()
        for line in lines:
            data = parse_annotation_line(line)
            data_dict = create_data_dict(data_fileds, data)
            writer.writerow(data_dict)